In [205]:
import pandas as pd

import numpy as np
import torch
from sklearn.model_selection import train_test_split
import tensorflow as tf
from matplotlib import pyplot as plt
import torch.nn as nn
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from collections import OrderedDict
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore")

EV_Grid = pd.DataFrame(pd.read_csv('annual_generation_WA.csv'))
x= np.arange(32) + 1990
dfy = EV_Grid
dfy['Year'] = np.arange(1990,2022)
dfy
Features=['Year', 'Total']
EV_Grid=EV_Grid[Features]
EV_Grid

,Year,Total
0,1990,1.020715e+08
1,1991,1.028706e+08
2,1992,8.648243e+07
3,1993,8.799578e+07
4,1994,8.858648e+07
5,1995,1.021976e+08
6,1996,1.186581e+08
7,1997,1.221262e+08
8,1998,1.021585e+08
9,1999,1.170840e+08


In [206]:
EV_Stations = pd.DataFrame(pd.read_csv('EV_stations.csv'))

# Filter the 'Latitude' column of the dataframe to only include values greater than 30
EV_Stations = EV_Stations.loc[EV_Stations['Latitude'] > 30]

EV_Stations = EV_Stations[EV_Stations['State']=='WA']
EV_Stations = EV_Stations['Open Date']
EV_Stations

51405    1/15/18
51406     3/1/10
51407    8/15/09
51408    4/15/10
51409    4/15/10
          ...   
53236    8/20/22
53237    8/23/22
53238    8/23/22
53239    8/23/22
53240    8/23/22
Name: Open Date, Length: 1835, dtype: object

In [207]:
EV_Vehicles = pd.DataFrame(pd.read_csv('Electric_Vehicle_Title_and_Registration_Activity.csv'))
EV_Vehicles = EV_Vehicles[EV_Vehicles['Transaction Type'] == 'Original Title']
EV_Vehicles.drop_duplicates(subset = "VIN (1-10)",keep = False, inplace= True) ##dropping duplicate vins
Features = ['DOL Transaction Date']
EV_Vehicles=EV_Vehicles[Features]
EV_Vehicles

,DOL Transaction Date
777,February 22 2022
950,August 16 2022
2079,March 25 2022
2264,November 23 2022
2308,December 13 2011
...,...
630896,June 18 2020
631084,September 14 2022
631417,May 28 2021
632255,December 04 2019


In [208]:
EV_Grid

,Year,Total
0,1990,1.020715e+08
1,1991,1.028706e+08
2,1992,8.648243e+07
3,1993,8.799578e+07
4,1994,8.858648e+07
5,1995,1.021976e+08
6,1996,1.186581e+08
7,1997,1.221262e+08
8,1998,1.021585e+08
9,1999,1.170840e+08


In [209]:
#def transform_total(total):
#    if str(total)[-2:] == '08':
#        return total * np.exp(8)
#    elif str(total)[-2:] == '07':
#        return total * np.exp(7)
#    else:
#        return total

#ev_grid_scaled['Total'] = ev_grid_scaled['Total'].apply(transform_total)

In [210]:
#ev_grid_scaled

In [211]:
#EV_Grid=ev_grid_scaled
print(EV_Vehicles.shape)
print(EV_Stations.shape)
print(EV_Grid.shape)

(1447, 1)
(1835,)
(32, 2)


In [212]:
print(EV_Stations)
print(EV_Grid)
print(EV_Vehicles)

51405    1/15/18
51406     3/1/10
51407    8/15/09
51408    4/15/10
51409    4/15/10
          ...   
53236    8/20/22
53237    8/23/22
53238    8/23/22
53239    8/23/22
53240    8/23/22
Name: Open Date, Length: 1835, dtype: object
    Year         Total
0   1990  1.020715e+08
1   1991  1.028706e+08
2   1992  8.648243e+07
3   1993  8.799578e+07
4   1994  8.858648e+07
5   1995  1.021976e+08
6   1996  1.186581e+08
7   1997  1.221262e+08
8   1998  1.021585e+08
9   1999  1.170840e+08
10  2000  1.082369e+08
11  2001  8.304867e+07
12  2002  1.027650e+08
13  2003  1.000947e+08
14  2004  1.021651e+08
15  2005  1.019658e+08
16  2006  1.082032e+08
17  2007  1.069902e+08
18  2008  1.108285e+08
19  2009  1.044701e+08
20  2010  1.034727e+08
21  2011  1.152626e+08
22  2012  1.168355e+08
23  2013  1.141729e+08
24  2014  1.163344e+08
25  2015  1.092875e+08
26  2016  1.140866e+08
27  2017  1.159121e+08
28  2018  1.167567e+08
29  2019  1.064636e+08
30  2020  1.161145e+08
31  2021  1.108084e+08
       DO

In [213]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.neural_network import MLPRegressor

# Load the datasets
vehicle_df = EV_Vehicles
station_df = EV_Stations
power_df =EV_Grid

# Convert the dates in the vehicle dataset to DDMMYYYY format
vehicle_df['DOL Transaction Date'] = pd.to_datetime(vehicle_df['DOL Transaction Date'], format='%B %d %Y').dt.strftime('%d%m%Y')

# Convert the dates in the station dataset to DDMMYYYY format
station_df = pd.to_datetime(station_df, format='%m/%d/%y').dt.strftime('%d%m%Y')

station_df = pd.DataFrame(station_df)
vehicle_df = pd.DataFrame(vehicle_df)

print(station_df)
print(vehicle_df)
print(power_df)

      Open Date
51405  15012018
51406  01032010
51407  15082009
51408  15042010
51409  15042010
...         ...
53236  20082022
53237  23082022
53238  23082022
53239  23082022
53240  23082022

[1835 rows x 1 columns]
       DOL Transaction Date
777                22022022
950                16082022
2079               25032022
2264               23112022
2308               13122011
...                     ...
630896             18062020
631084             14092022
631417             28052021
632255             04122019
633818             13042020

[1447 rows x 1 columns]
    Year         Total
0   1990  1.020715e+08
1   1991  1.028706e+08
2   1992  8.648243e+07
3   1993  8.799578e+07
4   1994  8.858648e+07
5   1995  1.021976e+08
6   1996  1.186581e+08
7   1997  1.221262e+08
8   1998  1.021585e+08
9   1999  1.170840e+08
10  2000  1.082369e+08
11  2001  8.304867e+07
12  2002  1.027650e+08
13  2003  1.000947e+08
14  2004  1.021651e+08
15  2005  1.019658e+08
16  2006  1.082032e+08
17  2007

In [223]:
# Combine the two dataframes using merge
df = pd.merge(station_df, vehicle_df, left_on='Open Date', right_on='DOL Transaction Date')

# Concatenate the dataframes along the rows (axis=0)
#df = pd.concat([station_df, vehicle_df], axis=0)
df.columns=['Stations','Vehicles']
df

,Stations,Vehicles
0,15072019,15072019
1,17092014,17092014
2,07042015,07042015
3,09042015,09042015
4,11082015,11082015
...,...,...
1570,23082022,23082022
1571,23082022,23082022
1572,23082022,23082022
1573,23082022,23082022


In [234]:
# Extract the features (X) and target (y)
X = df[['Stations', 'Vehicles']].values
y = power_df['Year']
print(y)
X

0     1990
1     1991
2     1992
3     1993
4     1994
5     1995
6     1996
7     1997
8     1998
9     1999
10    2000
11    2001
12    2002
13    2003
14    2004
15    2005
16    2006
17    2007
18    2008
19    2009
20    2010
21    2011
22    2012
23    2013
24    2014
25    2015
26    2016
27    2017
28    2018
29    2019
30    2020
31    2021
Name: Year, dtype: int32


array([['15072019', '15072019'],
       ['17092014', '17092014'],
       ['07042015', '07042015'],
       ...,
       ['23082022', '23082022'],
       ['23082022', '23082022'],
       ['23082022', '23082022']], dtype=object)

In [ ]:
import pandas as pd
import datetime

# Load the three datasets into pandas DataFrames
power_grid_df = EV_Grid
vin_date_df = EV_Vehicles
charging_station_df = EV_Stations

# Split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compare the predictions to the true values
from sklearn.metrics import mean_absolute_error

# Compile the model
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mean_absolute_error"])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Print the training and test accuracy and loss every 10 epochs
for epoch in range(100):
    if epoch % 10 == 0:
        train_acc = history.history["mean_absolute_error"][epoch]
        test_acc = history.history["val_mean_absolute_error"][epoch]
        train_loss = history.history["loss"][epoch]
        test_loss = history.history["val_loss"][epoch]
        print(f"Epoch {epoch}: train_acc = {train_acc}, test_acc = {test_acc}, train_loss = {train_loss}, test_loss = {test_loss}")

# Evaluate the model on the test data
y_pred = model.predict(X_test)
        
# Calculate the average power draw per month of a single car for each month in the test set
average_power_draw = y_pred / X_test["number_of_cars"]